In [44]:
! pip install pandas==2.3.3


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [45]:
import pandas as pd
from pathlib import Path

In [46]:
BASE_DIR = Path.cwd().parent      # .../AUTOPRICE-IQ/ETL
FILE1 = BASE_DIR / 'data' / 'leboncoin.csv'     # .../AUTOPRICE-IQ/ETL/data/leboncoin.csv

In [47]:
df_leboncoin = pd.read_csv(FILE1)
df_leboncoin.head()

,id,title,price_eur,year,kilometers,fuel,gearbox,location
0,0,Renault Clio 1.0 TCe 90ch Equilibre,15499,2023,23889,Essence,Manuelle,Metz 57000
1,1,BMW SERIE 4 GRAN COUPE F36 420d 190ch BVA8 Lou...,15490,2017,182000,Diesel,Automatique,Béziers 34500
2,2,Audi A3 TFSI,12900,2018,124,Essence,Manuelle,Heyrieux 38540
3,3,Lexus RX450h IV 313cv Luxe pack techno,26000,2017,168500,Hybride,Automatique,Bouc-Bel-Air 13320
4,4,Mercedes classe a,26000,2021,45144,Diesel,Automatique,Marseille 13012 12e Arrondissement


In [48]:
import re

In [49]:
allowed = re.compile(r'[^a-zA-Z0-9., ]+')

In [50]:
df_leboncoin["title"] = (
    df_leboncoin["title"]
        .astype(str)
        .apply(lambda x: allowed.sub("", x))
        .str.replace(r"\s+", " ", regex=True)
        .str.strip()
)
df_leboncoin.head()

,id,title,price_eur,year,kilometers,fuel,gearbox,location
0,0,Renault Clio 1.0 TCe 90ch Equilibre,15499,2023,23889,Essence,Manuelle,Metz 57000
1,1,BMW SERIE 4 GRAN COUPE F36 420d 190ch BVA8 Lou...,15490,2017,182000,Diesel,Automatique,Béziers 34500
2,2,Audi A3 TFSI,12900,2018,124,Essence,Manuelle,Heyrieux 38540
3,3,Lexus RX450h IV 313cv Luxe pack techno,26000,2017,168500,Hybride,Automatique,Bouc-Bel-Air 13320
4,4,Mercedes classe a,26000,2021,45144,Diesel,Automatique,Marseille 13012 12e Arrondissement


In [51]:
df_leboncoin["marque"] = (
    df_leboncoin["title"]
        .astype(str)
        .str.split()
        .apply(lambda mots: " ".join(mots[:3]))
)
df_leboncoin[["title", "marque"]].head()

,title,marque
0,Renault Clio 1.0 TCe 90ch Equilibre,Renault Clio 1.0
1,BMW SERIE 4 GRAN COUPE F36 420d 190ch BVA8 Lou...,BMW SERIE 4
2,Audi A3 TFSI,Audi A3 TFSI
3,Lexus RX450h IV 313cv Luxe pack techno,Lexus RX450h IV
4,Mercedes classe a,Mercedes classe a


In [52]:
df_leboncoin['fuel'].value_counts()

fuel
Diesel                  154
Essence                 134
Hybride                  29
Électrique               12
GPL                       4
Hybride Rechargeable      2
Autre                     1
Name: count, dtype: int64

In [53]:
df_leboncoin['gearbox'].value_counts()

gearbox
Manuelle       205
Automatique    131
Name: count, dtype: int64

In [54]:
df_leboncoin.head()

,id,title,price_eur,year,kilometers,fuel,gearbox,location,marque
0,0,Renault Clio 1.0 TCe 90ch Equilibre,15499,2023,23889,Essence,Manuelle,Metz 57000,Renault Clio 1.0
1,1,BMW SERIE 4 GRAN COUPE F36 420d 190ch BVA8 Lou...,15490,2017,182000,Diesel,Automatique,Béziers 34500,BMW SERIE 4
2,2,Audi A3 TFSI,12900,2018,124,Essence,Manuelle,Heyrieux 38540,Audi A3 TFSI
3,3,Lexus RX450h IV 313cv Luxe pack techno,26000,2017,168500,Hybride,Automatique,Bouc-Bel-Air 13320,Lexus RX450h IV
4,4,Mercedes classe a,26000,2021,45144,Diesel,Automatique,Marseille 13012 12e Arrondissement,Mercedes classe a


In [55]:
def split_location(loc):
    s = str(loc)
    m = re.search(r"(\d{5})", s)
    if not m:
        return pd.Series([s.strip(), None])
    
    ville = s[:m.start()].strip()
    cp = m.group(1)
    return pd.Series([ville, cp])

df_leboncoin[["ville", "code postale"]] = df_leboncoin["location"].apply(split_location)

df_leboncoin[["location", "ville", "code postale"]].head(10)

,location,ville,code postale
0,Metz 57000,Metz,57000
1,Béziers 34500,Béziers,34500
2,Heyrieux 38540,Heyrieux,38540
3,Bouc-Bel-Air 13320,Bouc-Bel-Air,13320
4,Marseille 13012 12e Arrondissement,Marseille,13012
5,Chaponost 69630,Chaponost,69630
6,Chaponost 69630,Chaponost,69630
7,Chaponost 69630,Chaponost,69630
8,Chaponost 69630,Chaponost,69630
9,Chaponost 69630,Chaponost,69630


In [56]:
df_leboncoin.head()

,id,title,price_eur,year,kilometers,fuel,gearbox,location,marque,ville,code postale
0,0,Renault Clio 1.0 TCe 90ch Equilibre,15499,2023,23889,Essence,Manuelle,Metz 57000,Renault Clio 1.0,Metz,57000
1,1,BMW SERIE 4 GRAN COUPE F36 420d 190ch BVA8 Lou...,15490,2017,182000,Diesel,Automatique,Béziers 34500,BMW SERIE 4,Béziers,34500
2,2,Audi A3 TFSI,12900,2018,124,Essence,Manuelle,Heyrieux 38540,Audi A3 TFSI,Heyrieux,38540
3,3,Lexus RX450h IV 313cv Luxe pack techno,26000,2017,168500,Hybride,Automatique,Bouc-Bel-Air 13320,Lexus RX450h IV,Bouc-Bel-Air,13320
4,4,Mercedes classe a,26000,2021,45144,Diesel,Automatique,Marseille 13012 12e Arrondissement,Mercedes classe a,Marseille,13012


In [57]:
df_leboncoin.dropna()

,id,title,price_eur,year,kilometers,fuel,gearbox,location,marque,ville,code postale
0,0,Renault Clio 1.0 TCe 90ch Equilibre,15499,2023,23889,Essence,Manuelle,Metz 57000,Renault Clio 1.0,Metz,57000
1,1,BMW SERIE 4 GRAN COUPE F36 420d 190ch BVA8 Lou...,15490,2017,182000,Diesel,Automatique,Béziers 34500,BMW SERIE 4,Béziers,34500
2,2,Audi A3 TFSI,12900,2018,124,Essence,Manuelle,Heyrieux 38540,Audi A3 TFSI,Heyrieux,38540
3,3,Lexus RX450h IV 313cv Luxe pack techno,26000,2017,168500,Hybride,Automatique,Bouc-Bel-Air 13320,Lexus RX450h IV,Bouc-Bel-Air,13320
4,4,Mercedes classe a,26000,2021,45144,Diesel,Automatique,Marseille 13012 12e Arrondissement,Mercedes classe a,Marseille,13012
...,...,...,...,...,...,...,...,...,...,...,...
331,331,307,1700,2005,283000,Diesel,Manuelle,Charancieu 38490,307,Charancieu,38490
332,332,Bmw 320d e90,13000,2006,150000,Diesel,Automatique,Le Mans 72000,Bmw 320d e90,Le Mans,72000
333,333,Seat 6j 1.2 tdi,2000,2010,110000,Diesel,Manuelle,Saint-Joseph 97212,Seat 6j 1.2,Saint-Joseph,97212
334,334,Fiat 500,6500,2015,91000,Essence,Manuelle,Saint-Génis-des-Fontaines 66740,Fiat 500,Saint-Génis-des-Fontaines,66740


In [58]:
cols = ["price_eur", "year", "kilometers"]

for c in cols:
    df_leboncoin[c] = pd.to_numeric(df_leboncoin[c], errors="coerce")

In [59]:
df_leboncoin.shape

(336, 11)

In [60]:
mask = df_leboncoin["marque"].str.contains(r"[A-Za-z]", regex=True, na=False)

df_leboncoin = df_leboncoin[mask].copy().reset_index(drop=True)

In [61]:
df_leboncoin.shape

(333, 11)

In [62]:
df_leboncoin.head()

,id,title,price_eur,year,kilometers,fuel,gearbox,location,marque,ville,code postale
0,0,Renault Clio 1.0 TCe 90ch Equilibre,15499,2023,23889,Essence,Manuelle,Metz 57000,Renault Clio 1.0,Metz,57000
1,1,BMW SERIE 4 GRAN COUPE F36 420d 190ch BVA8 Lou...,15490,2017,182000,Diesel,Automatique,Béziers 34500,BMW SERIE 4,Béziers,34500
2,2,Audi A3 TFSI,12900,2018,124,Essence,Manuelle,Heyrieux 38540,Audi A3 TFSI,Heyrieux,38540
3,3,Lexus RX450h IV 313cv Luxe pack techno,26000,2017,168500,Hybride,Automatique,Bouc-Bel-Air 13320,Lexus RX450h IV,Bouc-Bel-Air,13320
4,4,Mercedes classe a,26000,2021,45144,Diesel,Automatique,Marseille 13012 12e Arrondissement,Mercedes classe a,Marseille,13012


In [63]:
df_leboncoin["host"] = (df_leboncoin["marque"]
                        .astype(str)
                        .str.split()
                        .apply(lambda x: "".join(x[:1])))
df_leboncoin[["title", "marque", "host"]].head()

,title,marque,host
0,Renault Clio 1.0 TCe 90ch Equilibre,Renault Clio 1.0,Renault
1,BMW SERIE 4 GRAN COUPE F36 420d 190ch BVA8 Lou...,BMW SERIE 4,BMW
2,Audi A3 TFSI,Audi A3 TFSI,Audi
3,Lexus RX450h IV 313cv Luxe pack techno,Lexus RX450h IV,Lexus
4,Mercedes classe a,Mercedes classe a,Mercedes


In [64]:
df_leboncoin.head()

,id,title,price_eur,year,kilometers,fuel,gearbox,location,marque,ville,code postale,host
0,0,Renault Clio 1.0 TCe 90ch Equilibre,15499,2023,23889,Essence,Manuelle,Metz 57000,Renault Clio 1.0,Metz,57000,Renault
1,1,BMW SERIE 4 GRAN COUPE F36 420d 190ch BVA8 Lou...,15490,2017,182000,Diesel,Automatique,Béziers 34500,BMW SERIE 4,Béziers,34500,BMW
2,2,Audi A3 TFSI,12900,2018,124,Essence,Manuelle,Heyrieux 38540,Audi A3 TFSI,Heyrieux,38540,Audi
3,3,Lexus RX450h IV 313cv Luxe pack techno,26000,2017,168500,Hybride,Automatique,Bouc-Bel-Air 13320,Lexus RX450h IV,Bouc-Bel-Air,13320,Lexus
4,4,Mercedes classe a,26000,2021,45144,Diesel,Automatique,Marseille 13012 12e Arrondissement,Mercedes classe a,Marseille,13012,Mercedes


In [65]:
cols_order = [
    "id",
    "title",
    "marque",
    "host",
    "year",
    "kilometers",
    "price_eur",
    "fuel",
    "gearbox",
    "ville",
    "code postale",
    "location"
]
df_leboncoin = df_leboncoin[cols_order]
df_leboncoin.head()

,id,title,marque,host,year,kilometers,price_eur,fuel,gearbox,ville,code postale,location
0,0,Renault Clio 1.0 TCe 90ch Equilibre,Renault Clio 1.0,Renault,2023,23889,15499,Essence,Manuelle,Metz,57000,Metz 57000
1,1,BMW SERIE 4 GRAN COUPE F36 420d 190ch BVA8 Lou...,BMW SERIE 4,BMW,2017,182000,15490,Diesel,Automatique,Béziers,34500,Béziers 34500
2,2,Audi A3 TFSI,Audi A3 TFSI,Audi,2018,124,12900,Essence,Manuelle,Heyrieux,38540,Heyrieux 38540
3,3,Lexus RX450h IV 313cv Luxe pack techno,Lexus RX450h IV,Lexus,2017,168500,26000,Hybride,Automatique,Bouc-Bel-Air,13320,Bouc-Bel-Air 13320
4,4,Mercedes classe a,Mercedes classe a,Mercedes,2021,45144,26000,Diesel,Automatique,Marseille,13012,Marseille 13012 12e Arrondissement
